In [30]:
from collections import Counter
import datetime as dt 
import numpy as np
import os
import os.path
from os import path
import pandas as pd
import pickle
from sklearn import svm, neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from textblob import TextBlob

In [31]:
start = dt.datetime(2020,8,31)
end = dt.datetime(2020,9,1)

In [88]:
with open('sp100tickers.pickle','rb') as f:
    tickers = pickle.load(f)
joined_closes = pd.read_csv("sp100_joined_closes.csv")
dates = list(joined_closes["Date"])
#joined_closes.set_index("Date", inplace=True)
joined_closes

,Date,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
0,2020-08-28,124.807503,94.169998,110.790001,242.990005,516.440002,29.370001,94.300003,253.119995,249.589996,...,314.369995,195.130005,162.360001,37.200001,215.710007,59.259998,38.759998,24.690001,140.300003,40.689999
1,2020-08-31,129.039993,95.769997,109.470001,239.929993,513.390015,29.139999,93.000000,253.320007,249.149994,...,312.549988,192.440002,163.619995,36.400002,211.990005,59.270000,38.020000,24.150000,138.850006,39.939999
2,2020-09-01,134.179993,92.239998,106.550003,240.020004,527.950012,29.150000,92.959999,250.860001,248.029999,...,312.829987,195.800003,162.649994,36.689999,213.350006,59.160000,36.759998,24.049999,147.589996,39.430000
3,2020-09-02,131.399994,93.949997,109.440002,247.179993,533.799988,29.830000,94.589996,258.119995,256.500000,...,320.239990,196.279999,165.779999,37.610001,216.479996,60.529999,37.169998,24.570000,147.679993,39.189999
4,2020-09-03,120.879997,91.870003,105.529999,238.979996,507.799988,29.400000,93.339996,247.910004,252.320007,...,316.230011,190.550003,158.020004,37.459999,208.960007,60.610001,37.090000,24.520000,144.539993,39.110001
5,2020-09-04,120.959999,91.870003,104.160004,236.070007,491.940002,29.799999,93.809998,248.399994,249.979996,...,312.000000,191.139999,160.779999,38.310001,204.660004,60.480000,36.889999,24.790001,142.830002,39.080002


In [154]:
'''
I ended up using a modified version of this code later on, 
so I just commented out what I had
'''
# t_count = 0
# data_dict = {}
# for ticker in tickers:
#     d_count = 0
#     for date in dates:
#         if d_count == 0:
#             d_count += 1
#             continue
#         one = joined_closes.iloc[d_count - 1][ticker]
#         two = joined_closes.iloc[d_count][ticker]
#         change = (two - one) / one
#         data_dict[5 * t_count + d_count] = [date, ticker, change, float('NaN'), float('NaN')]
#         # data = data.append(pd.DataFrame(data={"Date": date, "Ticker": ticker, "Change": change, "Sentiment": 0, "Subjectivity": 0}, index=[t_count + d_count]), ignore_index=True)
#         d_count += 1
#     t_count += 1
# data = pd.DataFrame.from_dict(data_dict, orient='index', columns=["Date","Ticker", "Change", "Sentiment","Subjectivity"])
# data.to_csv("test_data.csv")
# data

,Date,Ticker,Change,Sentiment,Subjectivity
1,2020-08-31,AAPL,0.033912,NaN,NaN
2,2020-09-01,AAPL,0.039833,NaN,NaN
3,2020-09-02,AAPL,-0.020718,NaN,NaN
4,2020-09-03,AAPL,-0.080061,NaN,NaN
5,2020-09-04,AAPL,0.000662,NaN,NaN
...,...,...,...,...,...
501,2020-08-31,XOM,-0.018432,NaN,NaN
502,2020-09-01,XOM,-0.012769,NaN,NaN
503,2020-09-02,XOM,-0.006087,NaN,NaN
504,2020-09-03,XOM,-0.002041,NaN,NaN


In [155]:
'''
The Twitter api saves the creation date as a string with the date and time of creation.
I only care about the date, so I removed the time and saved the csv's as the same 
'''

def change_date(ticker):
    # A bug in my methods to scrape data does not save all tickers correctly.
    # For example, Bank of America (ticker BAC) is saved as "BA" and "C" for some reason.
    # I am just ignoring where these bugs occur
    if not os.path.exists("tweets/{}-combined.csv".format(ticker)):
        return
    tweets = pd.read_csv("tweets/{}-combined.csv".format(ticker))
    col_count = len(tweets.index)
    for i in range(len(tweets.index)):
        creation = tweets.iloc[i]["Creation"]
        creation = creation[:10]
        tweets.at[i, "Creation"] = creation
    tweets.to_csv("tweets/{}-combined.csv".format(ticker))
for ticker in tickers:
    change_date(ticker)

In [162]:
'''
This takes the tweets for a ticker at a certain date and returns the average sentiment
and subjectivity of for the day
'''
def get_sentiment(ticker, date):
    if not os.path.exists("tweets/{}-combined.csv".format(ticker)):
        return float('NaN'), float('NaN')
    tweets = pd.read_csv("tweets/{}-combined.csv".format(ticker))
    tweets_date = tweets.loc[tweets["Creation"] == date]
    count = 0 
    polarity = 0
    subjectivity = 0
    for tweet in tweets_date["Text"].tolist():
        sent = TextBlob(tweet)
        polarity += sent.polarity
        subjectivity += sent.subjectivity
        count += 1
    if count != 0:
        avg_pol = polarity / count
        avg_subj = subjectivity / count
        return avg_pol, avg_subj
    else:
        return float('NaN'), float('NaN')

    
# for date in dates[1:]:
#     count = 0
#     #tweets_date = tweets.loc[date]
#     tweets_date = tweets.loc[:,date]
#     print(tweets_date)
#     polarity = 0
#     subjectivity = 0
        


In [163]:
'''
This calculates daily change in price, and creates a dataframe containing dates, tickers, percentage change,
and sentiment analysis.  Saves to a file called test_data.csv
'''
t_count = 0
data_dict = {}
for ticker in tickers:
    d_count = 0
    for date in dates:
        if d_count == 0:
            d_count += 1
            continue
        one = joined_closes.iloc[d_count - 1][ticker]
        two = joined_closes.iloc[d_count][ticker]
        change = (two - one) / one
        pol, subj = get_sentiment(ticker, date)
        data_dict[5 * t_count + d_count] = [date, ticker, change, pol, subj]
        # data = data.append(pd.DataFrame(data={"Date": date, "Ticker": ticker, "Change": change, "Sentiment": 0, "Subjectivity": 0}, index=[t_count + d_count]), ignore_index=True)
        d_count += 1
    t_count += 1
data = pd.DataFrame.from_dict(data_dict, orient='index', columns=["Date","Ticker", "Change", "Sentiment","Subjectivity"])
data.to_csv("test_data.csv")

In [167]:
# Drops rows with unknown sentiment, as there was a bug in my data collection scripts.
data.dropna(inplace=True)
data

,Date,Ticker,Change,Sentiment,Subjectivity
1,2020-08-31,AAPL,0.033912,0.162516,0.331657
3,2020-09-02,AAPL,-0.020718,-0.090069,0.187037
4,2020-09-03,AAPL,-0.080061,0.142500,0.228333
5,2020-09-04,AAPL,0.000662,0.116667,0.252222
6,2020-08-31,ABBV,0.016991,0.141190,0.272857
...,...,...,...,...,...
500,2020-09-04,WMT,-0.011831,0.201556,0.218861
501,2020-08-31,XOM,-0.018432,0.174074,0.304966
503,2020-09-02,XOM,-0.006087,0.120000,0.211250
504,2020-09-03,XOM,-0.002041,0.125000,0.115625


In [190]:
'''
Creates labels for whether or not the stock moved enough in either direction.  Changing
the "cutoff" variable with change the labels.  Right now, a .7% increase or decrease created 
a model I was happy with, but if the portfolio prefers a different percentage change for buying or selling,
then this can easily be changed, and the rest of the code must be re-run.
'''
def buy_sell_hold():
    changes = data["Change"].tolist()
    cutoff = .007
    for c in range(len(changes)):
        if changes[c] > cutoff:
            changes[c] = 1
        elif changes[c] < -cutoff:
            changes[c] = -1
        else:
            changes[c] = 0
    return changes
changes = buy_sell_hold()
data["Labels"] = changes

In [191]:
# Checking the split of buy/sells/holds. Was hoping to get roughly 1/3, 1/3, 1/3 to be somewhat market neutral.
Counter(changes)

Counter({1: 108, -1: 138, 0: 81})

In [192]:
'''
ACTUALLY DOING LEARNING!!!!
'''

X = data[["Sentiment", "Subjectivity"]]
y = data["Labels"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .15)

# Using a few different classification algorithms, and selecting the best one.  
clf = VotingClassifier([('lsvc', svm.LinearSVC()), ('knn',neighbors.KNeighborsClassifier()), ('rfor', RandomForestClassifier())])

clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
    
print('Accuracy:', confidence)

predictions = clf.predict(X_test)
print('Predicted spread:', Counter(predictions))

Accuracy: 0.48
Predicted spread: Counter({-1: 41, 1: 5, 0: 4})


In [194]:
# Saving Model
pickle.dump(clf, open("finalized_model.pickle", "wb"))